In [31]:
#install the library on your host machine using: pip install bls (in conda if conda is interpreter)
#import libraries
import pandas as pd
import matplotlib as plt
import numpy as np
import bls

In [ ]:
#wages = bls.get_series('CIU2020000000000A') #get wagese data
#wages.to_csv('wages_data.csv') #write it to csv file
wages_data = pd.read_csv('wages_data.csv')
wages_data.columns = 'Date', 'Wages'
wages_data.head()

#wages show the annual average growth in %..
#e.g. 2013Q1 1.7% means if the trend continued it would be annul average growth

In [ ]:
#cpi = bls.get_series('CUUR0000SA0') #get cpi data
#cpi.to_csv('cpi_data.csv') #write it to csv
cpi_data = pd.read_csv('cpi_data.csv')
cpi_data.columns = 'Date', 'CPI'
cpi_data.head()

In [ ]:
#clean and manipulate data as wages are monthly and cpi quarterly measured
#along with wages shown in percentages, while the cpi is points

cpi_data['Date'] = cpi_data['Date'].str.replace('-03', 'Q1') #replace each instance -03 in Date colum with Q1
cpi_data['Date'] = cpi_data['Date'].str.replace('-06', 'Q2')
cpi_data['Date'] = cpi_data['Date'].str.replace('-09', 'Q3')
cpi_data['Date'] = cpi_data['Date'].str.replace('-12', 'Q4')

cpi_data = cpi_data[cpi_data.Date.str.contains('-') == False] #delete other rows containing '-'

cpi_data.head()

In [ ]:
##wages - divide each quarterly growth rate by four. This would be gross simplification of the process
# taking presumtion that the real changes to wages during those three months were exactly one quarter of the reported year-over-year rate. 
#determine baseline for calculation, that is 1st cpi value from the dataframe

#wagesam = bls.get_series('CIU2020000000000A') #read rata
#wagesam.to_csv('bls_wages_data.csv') #write to csv
df = pd.read_csv('bls_wages_data.csv') #read csv and save to dataframe
df.columns = 'Date','Wages' #rename columns

newdf = df 
newnum = pd.read_csv('cpi_data.csv')
newnum= newnum['CUUR0000SA0'][0] #assign starting CPI as baseline value
def process_wages(x):
    global newnum
    if type(x) is str:
        return x
    elif x:
        newnum = (x / 400) * newnum + newnum #convert wages from percentages to points and multiply by baseline to get approximative CPI
        return newnum
    else:
        return
newwages = newdf.applymap(process_wages)

newwages.head()

In [ ]:
#merge the data
merged_data = pd.merge(cpi_data, newwages, on='Date')
merged_data

In [ ]:
#plot the data
ax = merged_data.plot(kind='bar', figsize=(20, 7))
ax.set_xticklabels(merged_data.Date, rotation=45)
ax.set_ylabel('CPI vs. Wages and salaries - 12-month change')
ax.set_xlabel('Dates')

In [ ]:
ax = merged_data.plot(kind='line', figsize=(20, 7))
ax.set_ylabel('CPI vs. Wages and salaries - 12-month change')
ax.set_xlabel('Dates')
ax.grid()

In [ ]:
#alternative plots
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=merged_data['Date'], 
                                y=merged_data['CPI'], 
                                name = 'CPI'
                                ))
fig.add_trace(go.Scatter(x=merged_data['Date'], y=merged_data['Wages'], mode='lines', name='Approximated CPI'))
fig.update_layout(title='Consumer Price Index vs Approximated Consumer Prices Index based on Monthly Wages',
                   xaxis_title='Quarter',
                   yaxis_title='CPI / Approximated CPI')
fig.show()

In [32]:
#Incorporate S&P 500 quotes
import yfinance as yf
#data = yf.Ticker("^GSPC").history(period='10y')
#data.to_csv('new_s_p_500.csv') #write to csv
sp = pd.read_csv('new_s_p_500.csv')
sp.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2012-08-17,1415.839966,1418.709961,1414.670044,1418.160034,2922990000,0,0
1,2012-08-20,1417.849976,1418.130005,1412.119995,1418.130005,2766320000,0,0
2,2012-08-21,1418.130005,1426.680054,1410.859985,1413.170044,3282950000,0,0
3,2012-08-22,1413.089966,1416.119995,1406.780029,1413.489990,3062690000,0,0
4,2012-08-23,1413.489990,1413.489990,1400.500000,1402.079956,3008240000,0,0


In [33]:
#set Date column to use the datetime64 format. 
#use strftime to organize the dates so the year comes first, followed by month and day
#set the dash character - as the delimiter.

sp['Date'] = sp['Date'].astype('datetime64[ns]')
sp['Date'] = sp['Date'].dt.strftime('%Y-%m-%d')

In [34]:
sp = sp[sp.Date.str.contains('2012-') == False] #delete rows with 2012

In [ ]:
sp['Date'] = sp['Date'].str.replace('-03-30', 'Q1')
sp['Date'] = sp['Date'].str.replace('-06-30', 'Q2')
sp['Date'] = sp['Date'].str.replace('-09-30', 'Q3')
sp['Date'] = sp['Date'].str.replace('-12-31', 'Q4')

sp = sp[(sp.Date.str.contains("Q") == True)] #delete rows without 'Q'

In [ ]:
sp1 = sp[['Date', 'Close']]
sp1.head()

In [ ]:
#new dataframe made up of each of our three existing sets
from functools import reduce
dfs = [cpi_data, newwages, sp1]

df_triple = reduce(lambda left,right: pd.merge(left,right,on='Date'), dfs)

In [ ]:
ax = df_triple.plot(kind='line', figsize=(20, 7))
ax.set_ylabel('S&P, CPI, and Wages')
ax.set_xlabel('Dates')
ax.grid()

In [ ]:
ax = sp1.plot(kind='bar', figsize=(20, 7))
ax.set_xticklabels(sp1.Date, rotation=45)
ax.set_ylabel('S&P Index quotes by month')
ax.set_xlabel('Dates')

In [ ]:
ax = merged_data.plot(kind='line', figsize=(20, 7))
ax.set_xticklabels(merged_data.Date, rotation=45)
ax.set_ylabel('CPI vs. Wages and salaries - 12-month change')
ax.set_xlabel('Dates')

In [ ]:
#alternative plots
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1)

fig.append_trace(go.Scatter(x=merged_data['Date'], 
                                y=merged_data['CPI'], 
                                name = 'CPI'), row=1, col=1)

fig.add_trace(go.Scatter(x=merged_data['Date'], 
                         y=merged_data['Wages'], 
                         mode='lines', 
                         name='Approximated CPI'), row=1, col=1)

fig.append_trace(go.Scatter(x=sp1['Date'], 
                            y=sp1['Close'], 
                            mode='lines', 
                            name='S&P500'), row=2, col=1)

fig.update_layout(title='Consumer Price Index vs Wages vs S&P500', 
                  xaxis_title='Quarter', 
                  yaxis_title='CPI / Approximated CPI (Wages) / S&P500')
fig.show()


In [ ]:
#Calculate rate of increase for both CPI and S&P (CAGR - Compound Annual Growth Rate)

#first = 1681.550049 #first price
#last = 3785.379883  #last price
first = sp1['Close'].iat[0]
last = sp1['Close'].iat[-1]
periods = 10
    
def CAGR(first, last, periods):
    return (last/first)**(1/periods)-1

print ('The S&P had a CAGR of {:.2%} '.format(CAGR(first, last, periods)))

#The S&P had a CAGR of 8.45% 

In [ ]:
#first = 232.773 #first value
#last = 296.311  #last value
first = merged_data['CPI'].iat[0]
last = merged_data['CPI'].iat[-1] 
periods = 10
    
def CAGR(first, last, periods):
    return (last/first)**(1/periods)-1

print ('The Consumer Price Index had a CAGR of {:.2%} '.format(CAGR(first, last, periods)))

#The Consumer Price Index had a CAGR of 2.44%

In [ ]:
#first = 178.64
#last = 281.31
first = merged_data['Wages'].iat[0]
last = merged_data['Wages'].iat[-1]
periods = 20
    
def CAGR(first, last, periods):
    return (last/first)**(1/periods)-1

print ('Wages had a CAGR of {:.2%} '.format(CAGR(first, last, periods)))

#Wages had a CAGR of 1.32% 

So our consumer price index growth rate over the past 10 years was 2.449% and the employment wages rate was 1.32%. While, over the same time, the S&P 500 market index increased at an average annual rate of 8.45%!